# Data Crawling

In [44]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pyautogui as p
from openpyxl import Workbook

In [45]:
from selenium import webdriver
from selenium.webdriver import ChromeOptions
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from openpyxl import Workbook
from openpyxl import load_workbook

from bs4 import BeautifulSoup
import time

In [46]:
def set_chrome_driver():
    options = ChromeOptions()
    options.add_argument("--start-maximized")
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    return driver

## 솔드아웃 상품 목록, 상세 정보 Crawling

In [47]:
# 솔드아웃 상품 목록, 상세 정보 crawling
# 상위 몇개까지만 할지는 미정
# browser.back() 적용 아직 안함
browser = set_chrome_driver()
result = []

base_url = "https://www.soldout.co.kr"
url_soldout = base_url + "/search/product/list"
browser.get(url=url_soldout)

# 스크롤 내리기
interval = 3

prev_height = browser.execute_script("return document.body.scrollHeight")

while True:
    browser.execute_script("window.scrollTo(0,document.body.scrollHeight)")
    time.sleep(interval)
    cur_height = browser.execute_script("return document.body.scrollHeight")

    if cur_height == prev_height:
        break

    prev_height = cur_height
  

time.sleep(3)

soup = BeautifulSoup(browser.page_source, "lxml")

products = soup.find_all("div", class_="product-item")


products = products[:1000]
# 상품 목록 페이지에서 데이터 crawling
for info in products:
    # 상품 한글명
    name_kor = info.find("p", class_="product-name").text
    # 브랜드명
    brand = info.find("span", class_="brand-logo__text").text
    # 이미지 주소
    img_tag = info.find("img")
    img_url = img_tag["src"]
    # 상세페이지 주소
    product_detail_tag = info.find("a",class_="link-for-seo")
    # https://www.soldout.co.kr/trade/detail/5534466
    product_detail_url = base_url + product_detail_tag["href"]

    # 상품 상세 페이지에서 데이터 crawling
    product_detail_url_list = []
    product_detail_url_list.append(product_detail_url)

    for url in product_detail_url_list:
        browser.get(url=url)

        # 스크롤 내리기
        interval = 3
        prev_height = browser.execute_script("return document.body.scrollHeight")

        while True:
            browser.execute_script("window.scrollTo(0,document.body.scrollHeight)")
            time.sleep(interval)
            cur_height = browser.execute_script("return document.body.scrollHeight")

            if cur_height == prev_height:
                break

            prev_height = cur_height

        time.sleep(2)

        soup = BeautifulSoup(browser.page_source, "lxml")
       
        # 상품 영어명
        name_eng = soup.select_one("#__layout > div > div.layout-container > div > div.item-container__in > div.container-right > div.item_info__wrap > p")
        # 모델번호
        model_no = soup.select_one("#__layout > div > div.layout-container > div > div.item-container__in > div.container-right > div:nth-child(8) > dl:nth-child(3) > dd")
        # 출시일(DataFrame에서 날짜형태로 바꾸기)
        release_date = soup.select_one("#__layout > div > div.layout-container > div > div.item-container__in > div.container-right > div:nth-child(8) > dl:nth-child(4) > dd")
        # 색상
        color = soup.select_one("#__layout > div > div.layout-container > div > div.item-container__in > div.container-right > div:nth-child(8) > dl.product-info__dl.color > dd")
        # 원래 가격("원" 제거 후 DataFrame에서 int형으로 바꾸기)
        original_price = soup.select_one("#__layout > div > div.layout-container > div > div.item-container__in > div.container-right > div:nth-child(8) > dl:nth-child(6) > dd")

        if name_eng and model_no and release_date and color and original_price:
            name_eng = name_eng.text
            model_no = model_no.text
            release_date = release_date.text
            color = color.text
            original_price = original_price.text
        else:
            name_eng = np.nan
            model_no = np.nan
            release_date = np.nan
            color = np.nan
            original_price = np.nan

        # print(brand, name_kor, name_eng, model_no, release_date, color, original_price, img_url, product_detail_url)

    result.append([brand, name_kor, name_eng, model_no, release_date, color, original_price, img_url, product_detail_url])

columns = ["Brand", "Name_Kor", "Name_Eng", "Model_No", "Release_Date", "Color", "Original_Price", "Image_URL", "Product_Detail_URL"]
df = pd.DataFrame(result, columns=columns)
df.to_excel("./data/soldout_products2.xlsx", index=False)

wb = load_workbook("./data/soldout_products2.xlsx")
ws = wb.active

# 열의 너비 설정
ws.column_dimensions["A"].width = 20
ws.column_dimensions["B"].width = 100
ws.column_dimensions["C"].width = 100
ws.column_dimensions["D"].width = 50
ws.column_dimensions["E"].width = 50
ws.column_dimensions["F"].width = 50
ws.column_dimensions["G"].width = 50
ws.column_dimensions["H"].width = 100
ws.column_dimensions["I"].width = 60

# 최종 엑셀 파일로 저장
wb.save("./data/soldout_products2.xlsx")

print(len(products))
print("엑셀 파일로 저장 완료!")

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=126.0.6478.127)
Stacktrace:
	GetHandleVerifier [0x00CAC1C3+27395]
	(No symbol) [0x00C43DC4]
	(No symbol) [0x00B41B7F]
	(No symbol) [0x00B1E483]
	(No symbol) [0x00BAA06F]
	(No symbol) [0x00BBC3D6]
	(No symbol) [0x00BA3736]
	(No symbol) [0x00B77541]
	(No symbol) [0x00B780BD]
	GetHandleVerifier [0x00F63A93+2876371]
	GetHandleVerifier [0x00FB7F5D+3221661]
	GetHandleVerifier [0x00D2D634+556916]
	GetHandleVerifier [0x00D3474C+585868]
	(No symbol) [0x00C4CE04]
	(No symbol) [0x00C49818]
	(No symbol) [0x00C499B7]
	(No symbol) [0x00C3BF0E]
	BaseThreadInitThunk [0x76EBFCC9+25]
	RtlGetAppContainerNamedObjectPath [0x77D480CE+286]
	RtlGetAppContainerNamedObjectPath [0x77D4809E+238]


## 솔드아웃 거래내역 Crawling

In [ ]:
# 솔드아웃 로그인
browser = set_chrome_driver()
browser.get(url_soldout)
result2 = []

time.sleep(2)

login_button = browser.find_element(By.XPATH, '//*[@id="__layout"]/div/div[1]/header/div/ul/li[1]/a')
login_button.click()

time.sleep(2)

id_input = browser.find_element(By.CSS_SELECTOR, "#__layout > div > div.layout-container > div > form > div:nth-child(1) > div > input")
id_input.send_keys("uj05273")

pwd_input = browser.find_element(By.CSS_SELECTOR, "#__layout > div > div.layout-container > div > form > div:nth-child(2) > div > input")
pwd_input.send_keys("brian981103")

signin_button = browser.find_element(By.CLASS_NAME, "btn-primary")
signin_button.click()

time.sleep(2)

# 거래내역 crawling
for info in products[:100]:
    # 상세페이지 주소
    product_detail_tag = info.find("a",class_="link-for-seo")
    # https://www.soldout.co.kr/trade/detail/5534466
    product_detail_url = base_url + product_detail_tag["href"]

    # 상품 상세 페이지에서 데이터 crawling
    product_detail_url_list = []
    product_detail_url_list.append(product_detail_url)

    for url in product_detail_url_list:
        #print(url)
        browser.get(url=url)
        
        time.sleep(5)

        # 거래내역 전체보기 버튼 클릭
        view_all_btn = browser.find_element(By.CLASS_NAME, "btn-show-all")
        view_all_btn.click()

        time.sleep(3)

        # 상품 목록 데이터 중 한글이름과 merge 하기 위함
        name_kor = soup.find("p",class_="name_kor")
        if name_kor:
            name_kor = name_kor.text
        
        # 스크롤을 위해 마우스 중앙으로 옮기기
        p.moveTo(1270,815,0.5)
        p.click()

        modal_content = browser.find_element(By.CSS_SELECTOR, "body > div.trade_modal.BaseModal > div > div > div.base-table.trade_modal__table.modal-table > table > tbody")
        prev_height = browser.execute_script("return arguments[0].scrollHeight", modal_content)

        # 모달 창 스크롤 내리기
        while True:
            browser.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight;", modal_content)
            time.sleep(1)
            cur_height = browser.execute_script("return arguments[0].scrollHeight", modal_content)

            if cur_height == prev_height:
                break

            prev_height = cur_height
        
        soup = BeautifulSoup(browser.page_source, "lxml")
        trades = soup.select("body > div.trade_modal.BaseModal > div > div > div.base-table.trade_modal__table.modal-table > table > tbody > tr")
        
        for trade in trades:
            trade_dates = trade.select_one("tbody > tr > td:nth-child(1)")
            trade_sizes = trade.select_one("tbody > tr > td:nth-child(2)")
            trade_prices = trade.select_one("tbody > tr > td:nth-child(3) > span")

            if trade_dates and trade_sizes and trade_prices:
                trade_dates = trade_dates.text
                trade_sizes = trade_sizes.text
                trade_prices = trade_prices.text
        
            result2.append([name_kor,trade_dates,trade_sizes,trade_prices])


columns = ["Name_Kor", "Trade_Dates", "Trade_Sizes", "Trade_Prices"]
df = pd.DataFrame(result2, columns=columns)
df.to_excel("./data/soldout_trades2.xlsx", index=False)

wb = load_workbook("./data/soldout_trades2.xlsx")
ws = wb.active

# 열의 너비 설정
ws.column_dimensions["A"].width = 50
ws.column_dimensions["B"].width = 35
ws.column_dimensions["C"].width = 30
ws.column_dimensions["D"].width = 30

wb.save("./data/soldout_trades2.xlsx")

print(len(products))
print("엑셀 파일로 저장 완료!")
            
    # X버튼으로 나가기 -> 뒤로가기 버튼
    # x_btn = browser.find_element(By.XPATH,'/html/body/div[5]/div/header/div[3]/button')
    # x_btn.click()
    # browser.back()

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".btn-show-all"}
  (Session info: chrome=126.0.6478.127); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00DAC1C3+27395]
	(No symbol) [0x00D43DC4]
	(No symbol) [0x00C41B7F]
	(No symbol) [0x00C82C65]
	(No symbol) [0x00C82D3B]
	(No symbol) [0x00CBEC82]
	(No symbol) [0x00CA39E4]
	(No symbol) [0x00CBCB24]
	(No symbol) [0x00CA3736]
	(No symbol) [0x00C77541]
	(No symbol) [0x00C780BD]
	GetHandleVerifier [0x01063A93+2876371]
	GetHandleVerifier [0x010B7F5D+3221661]
	GetHandleVerifier [0x00E2D634+556916]
	GetHandleVerifier [0x00E3474C+585868]
	(No symbol) [0x00D4CE04]
	(No symbol) [0x00D49818]
	(No symbol) [0x00D499B7]
	(No symbol) [0x00D3BF0E]
	BaseThreadInitThunk [0x75D7FCC9+25]
	RtlGetAppContainerNamedObjectPath [0x774180CE+286]
	RtlGetAppContainerNamed

## Kream에서 상품 상세페이지, 상품코드 crawling

In [66]:
browser = set_chrome_driver()
base_url = "https://kream.co.kr"
url = base_url + "/search"
browser.get(url=url)

time.sleep(3)
result = []

# 스크롤 내리기
# interval = 3
# prev_height = browser.execute_script("return document.body.scrollHeight")

# while True:
#     browser.execute_script("window.scrollTo(0,document.body.scrollHeight)")
#     time.sleep(interval)
#     cur_height = browser.execute_script("return document.body.scrollHeight")

#     if cur_height == prev_height:
#         break

#     prev_height = cur_height

# time.sleep(4)

# 상품마다 각 상세 페이지 가져오기
soup = BeautifulSoup(browser.page_source, "lxml")
products = soup.find_all("div",class_="search_result_item product")



product_detail_url_list = []

for product in products:
    
    product_detail_url = product.find("a",class_="item_inner")["href"]
    # print("product_detail_url ",product_detail_url)
    product_detail_urls = base_url + product_detail_url
    
    # 상품 상세 페이지에서 데이터 crawling
    product_detail_url_list.append(product_detail_urls)

print(product_detail_url_list)


        

['https://kream.co.kr/products/307550', 'https://kream.co.kr/products/21935', 'https://kream.co.kr/products/12831', 'https://kream.co.kr/products/15251', 'https://kream.co.kr/products/247226', 'https://kream.co.kr/products/288640', 'https://kream.co.kr/products/28260', 'https://kream.co.kr/products/58293', 'https://kream.co.kr/products/244293', 'https://kream.co.kr/products/310709', 'https://kream.co.kr/products/67193', 'https://kream.co.kr/products/137241', 'https://kream.co.kr/products/34146', 'https://kream.co.kr/products/114316', 'https://kream.co.kr/products/304900', 'https://kream.co.kr/products/58291', 'https://kream.co.kr/products/15248', 'https://kream.co.kr/products/262947', 'https://kream.co.kr/products/297522', 'https://kream.co.kr/products/306357', 'https://kream.co.kr/products/219423', 'https://kream.co.kr/products/284030', 'https://kream.co.kr/products/36038', 'https://kream.co.kr/products/97779', 'https://kream.co.kr/products/90805', 'https://kream.co.kr/products/240723

In [68]:
for url in product_detail_url_list[5:6]:
    
    print("url {}".format(url))

    browser.get(url=url)
    soup = BeautifulSoup(browser.page_source, "lxml")

    model_no = soup.select_one("div.product_info_wrap > div > dl > div:nth-child(3) > div.product_info")
    if model_no:
        model_no = model_no.text.strip()
        print("model_no ",model_no)

    prouct_detail_url = url.split("/")
     = urls[0]+"/social"+

    # https://kream.co.kr/social/products/12831
    social_url = base_url + "/social" + product_detail_url
    print("social_url {}".format(social_url))

    browser.get(url=social_url)
    time.sleep(3)

    # 리뷰 전체를 보기위하여 스크롤 내리기
    interval = 3
    prev_height = browser.execute_script("return document.body.scrollHeight")
    while True:
        browser.execute_script("window.scrollTo(0,document.body.scrollHeight)")
        time.sleep(interval)
        cur_height = browser.execute_script("return document.body.scrollHeight")

        if cur_height == prev_height:
            break


        prev_height = cur_height

    time.sleep(3)

    soup = BeautifulSoup(browser.page_source, "lxml")
    socials = soup.find_all("div",class_="feed_card")
    print(len(socials))

    for social in socials:
        social_text = social.find("p",class_="text_box")
        if social_text:
            social_text = social_text.text
        else:
            social_text = np.nan
        result.append([model_no,social_text])



columns = ["Model_No","Social_Text"]
df = pd.DataFrame(result, columns=columns)
df.to_excel("./data/kream_social_text_text.xlsx", index=False)



print("엑셀 파일로 저장 완료!")

url https://kream.co.kr/products/288640
model_no  DZ4137-700
social_url https://kream.co.kr/social/products/26344
20
엑셀 파일로 저장 완료!


In [76]:
browser = set_chrome_driver()
social_url='https://kream.co.kr/social/products/247226'
browser.get(url=social_url)
time.sleep(3)

result = []

# 리뷰 전체를 보기위하여 스크롤 내리기
interval = 3
prev_height = browser.execute_script("return document.body.scrollHeight")
while True:
    browser.execute_script("window.scrollTo(0,document.body.scrollHeight)")
    time.sleep(interval)
    cur_height = browser.execute_script("return document.body.scrollHeight")

    if cur_height == prev_height:
        break

    prev_height = cur_height

time.sleep(3)

soup = BeautifulSoup(browser.page_source, "lxml")
# socials = soup.find_all("div",class_="feed_card")
# print(len(socials))

socials = soup.select("div.card_detail > p")
print(len(socials))

for social in socials:
    # social_text = social.find("p",class_="text_box")
    # if social_text:
    #     social_text = social_text.text        
    # else:
    #     social_text = np.nan
    print(social,social.string.strip())
    result.append([model_no,social.string.strip()])


columns = ["Model_No","Social_Text"]
df = pd.DataFrame(result, columns=columns)
df.to_excel("./data/kream_social_text_text.xlsx", index=False)

72
<p class="text_box" data-v-333cb905=""> 기본템으로 좋음 </p> 기본템으로 좋음
<p class="text_box" data-v-333cb905=""> #비오는날코디 #썸머룩 #여름스타일 #stussy </p> #비오는날코디 #썸머룩 #여름스타일 #stussy
<p class="text_box" data-v-333cb905=""> 1위제품들로만 구성한 커플룩! </p> 1위제품들로만 구성한 커플룩!
<p class="text_box" data-v-333cb905=""> #핫트렌드챌린지 #페스티벌룩 #썸머코디 #사이즈팁 #휴가준비 #스타일공유 #트렌드슈즈 #KICKS #여름데일리 #여름신발 </p> #핫트렌드챌린지 #페스티벌룩 #썸머코디 #사이즈팁 #휴가준비 #스타일공유 #트렌드슈즈 #KICKS #여름데일리 #여름신발
<p class="text_box" data-v-333cb905=""> 여름이라도 올블랙 </p> 여름이라도 올블랙
<p class="text_box" data-v-333cb905=""> 야레야레 못말리는 아가씨 </p> 야레야레 못말리는 아가씨
<p class="text_box" data-v-333cb905=""> #핫트렌드챌린지 #페스티벌룩 #썸머코디 #휴가준비 #스타일공유 #여름데일리 #ootd </p> #핫트렌드챌린지 #페스티벌룩 #썸머코디 #휴가준비 #스타일공유 #여름데일리 #ootd
<p class="text_box" data-v-333cb905=""> 스투시 </p> 스투시
<p class="text_box" data-v-333cb905=""> 헤헤 나도 스투시 샀당 </p> 헤헤 나도 스투시 샀당
<p class="text_box" data-v-333cb905=""> STÜSSY 👤 PÄTTA  BLK </p> STÜSSY 👤 PÄTTA  BLK
<p class="text_box" data-v-333cb905=""> 스투시xl 성공 184 90 </p> 스투시xl 성공 184 90
<p class